# <a href="https://girafe.ai/" target="_blank" rel="noopener noreferrer"><img src="https://raw.githubusercontent.com/girafe-ai/ml-course/7096a5df4cada5ee651be1e3215c2f7fb8a7e0bf/logo_margin.svg" alt="girafe-ai logo" width="150px" align="left"></a> [ml-basic course](https://github.com/girafe-ai/ml-course) <a class="tocSkip">

# Lab assignment №1, part 2
## Gradient boosting on temporal data and feature importances

Today we will work with Gradient Boosting library. It is one of the most popular models these days that shows both great quality and performance.

Choises for library are:

* [LightGBM](https://github.com/Microsoft/LightGBM) by Microsoft. Handful and fast.
* [Catboost](https://github.com/catboost/catboost) by Yandex. Tuned to deal well with categorical features.
* [xgboost](https://github.com/dmlc/xgboost) by dlmc. The most famous framework which got very popular on kaggle.

**Dataset**

By default we will work with widely known [Human Actividy Recognition (HAR) dataset](https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones). Data is available at UCI repository.

There are available both raw and preprocessed datasets. This time we will use the preprocessed one.
Some simple preprocessing is done for you.

If you want more interpretable data, you can take [Wine quality dataset](https://archive.ics.uci.edu/dataset/186/wine+quality) (see details below).

Your __ultimate target is to get familiar with one of the frameworks above__ and achieve at least 90% accuracy on test dataset and try to get some useful insights on the features the model paid attention to.

_Despite the main language of this notebook is English, feel free to write your thoughts in Russian._

## Part 0. Downloading and preprocessing

The preprocessing is done for you. Let's take a look at the data:

In [ ]:
# Download and unpack dataset from UCI
!wget -nc https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip
!unzip -u "UCI HAR Dataset.zip" "UCI HAR Dataset/train/X_train.txt" "UCI HAR Dataset/train/y_train.txt" \
"UCI HAR Dataset/test/X_test.txt" "UCI HAR Dataset/test/y_test.txt" "UCI HAR Dataset/activity_labels.txt"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
X_train = np.genfromtxt("UCI HAR Dataset/train/X_train.txt")
y_train = np.genfromtxt("UCI HAR Dataset/train/y_train.txt")
print(f"Train set: {X_train.shape}, {y_train.shape}")

X_test = np.genfromtxt("UCI HAR Dataset/test/X_test.txt")
y_test = np.genfromtxt("UCI HAR Dataset/test/y_test.txt")
print(f"Test set: {X_test.shape}, {y_test.shape}")

n_features = X_train.shape[1]

In [ ]:
activity_labels = {}
with open("UCI HAR Dataset/activity_labels.txt", "r") as file:
    for line in file:
        label, name = line.strip().split(" ")
        activity_labels[int(label)] = name

activity_labels

Let's normalize data

In [ ]:
data_mean = X_train.mean(axis=0)
data_std = X_train.std(axis=0)

X_train = (X_train - data_mean) / data_std
X_test = (X_test - data_mean) / data_std

The dataset has some duplicating features. Let's remove them

In [ ]:
# fmt: off
duplicating_columns = (
    205, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 231, 244, 257, 507, 520, 533, 546,
)
# fmt: on

duplicating_mask = np.isin(range(n_features), duplicating_columns)

In [ ]:
X_train_unique = X_train[:, ~duplicating_mask]
X_test_unique = X_test[:, ~duplicating_mask]

X_train_unique.shape, X_test_unique.shape

PCA could be useful in this case. E.g.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(0.99)

X_train_pca = pca.fit_transform(X_train_unique)
X_test_pca = pca.transform(X_test_unique)

X_train_pca.shape, X_test_pca.shape

In [ ]:
plt.scatter(X_train_pca[:1000, 0], X_train_pca[:1000, 1], c=y_train[:1000])
plt.grid()
plt.xlabel("Principal component 1")
plt.ylabel("Principal component 2")

In [ ]:
plt.scatter(X_train_pca[:1000, 3], X_train_pca[:1000, 4], c=y_train[:1000])
plt.grid()
plt.xlabel("Principal component 4")
plt.ylabel("Principal component 5")

### Alternative dataset: Wine quality

Please, take this dataset if you are sure you can preprocess it yourself and ready to work with it's features and results, so it is done on your risk.

However you will have interpretable features which can be analysed with shap in last part

In [ ]:
!pip install ucimlrepo

In [ ]:
import ucimlrepo as uci

In [ ]:
dataset = uci.fetch_ucirepo(id=186)

print(dataset.metadata.name, '\n')
print(dataset.metadata.abstract, '\n')
print(dataset.metadata.additional_info.summary, '\n')

## Part 1. Fit the model.

Despite optimal parameters (e.g. for xgboost) can be found on the web, we still want you to approximate them by yourself.

In this part just check some (3-5) sets of hyperparameters by hand.

In [ ]:
!pip install catboost

In [ ]:
# Example: https://rpubs.com/burakh/har_xgb

from sklearn.metrics import accuracy_score

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

Напишем Функцию для тренировки модели и вывода метрик:

In [ ]:
def train_and_evaluate(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    return acc

### **LightGBM**

In [ ]:
print("Testing LightGBM with different hyperparameters:")

lgbm_params_list = [
    {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 7, "random_state": 42},
    {"n_estimators": 150, "learning_rate": 0.05, "max_depth": 9, "random_state": 42},
    {"n_estimators": 200, "learning_rate": 0.1, "max_depth": 5, "random_state": 42},
    {"n_estimators": 300, "learning_rate": 0.05, "max_depth": 6, "random_state": 42},
    {"n_estimators": 250, "learning_rate": 0.2, "max_depth": 4, "random_state": 42},
]

for i, params in enumerate(lgbm_params_list, 1):
    print(f"\nSet {i}: {params}")

    lgbm_model = LGBMClassifier(**params, verbose=-1)
    acc = train_and_evaluate(lgbm_model, X_train_pca, y_train, X_test_pca, y_test)

    print(f"Accuracy: {acc:.4f}")

### **CatBoost**

In [ ]:
print("Testing CatBoost with different hyperparameters:")

catboost_params_list = [
    {"iterations": 100, "learning_rate": 0.1, "depth": 7, "verbose": False, "random_seed": 42},
    {"iterations": 150, "learning_rate": 0.05, "depth": 9, "verbose": False, "random_seed": 42},
    {"iterations": 200, "learning_rate": 0.1, "depth": 5, "verbose": False, "random_seed": 42},
    {"iterations": 300, "learning_rate": 0.05, "depth": 6, "verbose": False, "random_seed": 42},
    {"iterations": 250, "learning_rate": 0.2, "depth": 4, "verbose": False, "random_seed": 42},
]

for i, params in enumerate(catboost_params_list, 1):
    print(f"\nSet {i}: {params}")

    catboost_model = CatBoostClassifier(**params)
    acc = train_and_evaluate(catboost_model, X_train_pca, y_train, X_test_pca, y_test)

    print(f"Accuracy: {acc:.4f}")

### **XGBoost**

In [ ]:
import xgboost as xgb

In [ ]:
print("Testing XGBoost with different hyperparameters:")

xgboost_params_list = [
    {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 7, "random_state": 42, "eval_metric": "logloss"},
    {"n_estimators": 150, "learning_rate": 0.05, "max_depth": 9, "random_state": 42, "eval_metric": "logloss"},
    {"n_estimators": 200, "learning_rate": 0.1, "max_depth": 5, "random_state": 42, "eval_metric": "logloss"},
    {"n_estimators": 300, "learning_rate": 0.05, "max_depth": 6, "random_state": 42, "eval_metric": "logloss"},
    {"n_estimators": 250, "learning_rate": 0.2, "max_depth": 4, "random_state": 42, "eval_metric": "logloss"},
]

dtrain = xgb.DMatrix(X_train_pca, label=y_train)
dtest = xgb.DMatrix(X_test_pca, label=y_test)

for i, params in enumerate(xgboost_params_list, 1):
    print(f"\nSet {i}: {params}")

    num_rounds = params.pop("n_estimators")
    model = xgb.train(params, dtrain, num_boost_round=num_rounds)

    y_pred = model.predict(dtest)
    y_pred = [int(pred) for pred in y_pred]
    accuracy = accuracy_score(y_test, y_pred)

    print(f"Accuracy: {accuracy:.4f}")

**Выводы:**

  - **LightGBM**:
  
  Наилучший результат показал Набор 5 (n_estimators=250, learning_rate=0.2, max_depth=4), с точностью 0.9287.

  - **CatBoost:**
  
  Наилучший результат показал также Набор 5 (iterations=250, learning_rate=0.2, depth=4), с точностью 0.9277.

  - **XGBoost:**
  
  Точность XGBoost значительно ниже, чем у LightGBM и CatBoost, и наилучший результат в Наборе 5 составляет 0.5402.

**Оптимальная модель**:

Из всех моделей LightGBM (Набор 5) показал наилучшие результаты с точностью 0.9287, что делает его наиболее подходящим для дальнейшего использования.

## Part 2. Use hyper parameter tuning system

Use [optuna](https://optuna.org/), [hyperopt](http://hyperopt.github.io/hyperopt/) or any other zero order optimizer to find optimal hyper param set.

In [ ]:
!pip install optuna

In [ ]:
from sklearn.model_selection import train_test_split
import optuna
import lightgbm as lgb
import pandas as pd

In [ ]:
X_train_opt, X_val_opt, y_train_opt, y_val_opt = train_test_split(X_train_pca, y_train, test_size=0.2, random_state=42)

# Функция оптимизации для Optuna
def objective(trial):
    params = {
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 31, 256),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    }

    model = lgb.LGBMClassifier(**params, verbose=-1)
    model.fit(X_train_opt, y_train_opt)

    y_pred = model.predict(X_val_opt)
    accuracy = accuracy_score(y_val_opt, y_pred)

    return accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print(f"Best parameters: {study.best_params}")
print(f"Best accuracy: {study.best_value:.4f}")

df_trials = pd.DataFrame(study.trials_dataframe())
df_trials_sorted = df_trials.sort_values(by="value", ascending=False)

print("\nDetailed results of all trials (sorted by accuracy):")
print(df_trials_sorted[['number', 'value',
                        'params_num_leaves',
                        'params_max_depth',
                        'params_learning_rate',
                        'params_n_estimators',
                        'params_min_child_samples',
                        'params_subsample',
                        'params_colsample_bytree']].head(10))

### Conclusion

Please, write down your thoughts on the experiment results:


**Лучшие параметры:**  
- `num_leaves`: **117**  
- `max_depth`: **4**  
- `learning_rate`: **0.1533**  
- `n_estimators`: **364**  
- `min_child_samples`: **89**  
- `subsample`: **0.7953**  
- `colsample_bytree`: **0.5286**  

**Лучшая accuracy:** **97.55%**

**Вывод:**

Модель показала высокую accuracy, что свидетельствует о хорошем подборе гиперпараметров. Умеренная глубина деревьев и размер листьев позволили избежать переобучения, а комбинация параметров `learning_rate` и `n_estimators` обеспечила баланс между скоростью обучения и качеством.

## Part 3. Interpret the model predictions

Please use [shap](https://github.com/slundberg/shap) to build some plots and try to interpret them.

In [ ]:
!pip install shap

In [ ]:
import shap  # noqa: F401

Сохраним лучшую модель из части 2:

In [ ]:
best_model_params = study.best_params
best_model = lgb.LGBMClassifier(**best_model_params, verbose=-1)
best_model.fit(X_train_opt, y_train_opt)

SHAP Explainer:

In [ ]:
explainer = shap.Explainer(best_model)
shap_values = explainer.shap_values(X_test_pca)

Проверим количество классов:

In [ ]:
num_classes = shap_values.shape[1] if len(shap_values.shape) > 2 else 1
print("Распознанное количество классов:", num_classes)

shap_values_mean = shap_values.mean(axis=2)

Глобальная важность признаков:

In [ ]:
print("Summary Plot (Bar) - Averaged SHAP values across classes")
shap.summary_plot(shap_values_mean, X_test_pca, plot_type="bar")
plt.show()

Распределение значений SHAP:

In [ ]:
print("Summary Plot (Distribution) - Averaged SHAP values across classes")
shap.summary_plot(shap_values_mean, X_test_pca)
plt.show()

Влияние конкретного признака на предсказания:

In [ ]:
feature_to_plot = 0  # Можно заменить на индекс интересующего признака
print(f"Dependence Plot - Feature {feature_to_plot} - Averaged SHAP values")
shap.dependence_plot(feature_to_plot, shap_values_mean, X_test_pca)
plt.show()

Индивидуальное объяснение для одного примера:

In [ ]:
sample_index = 0  # Можно заменить на индекс интересующего признака
print(f"Force Plot for example {sample_index} - Averaged SHAP values")
shap.initjs()
expected_value_mean = np.mean(explainer.expected_value)
shap.force_plot(expected_value_mean, shap_values_mean[sample_index], X_test_pca[sample_index])

### Conclusion

Your thoughts about the plots and model behaviour:

1. **Ключевые признаки**:  
   Feature 0 оказался самым значимым для модели, с большим отрывом по важности. За ним следуют Feature 3, Feature 4, Feature 1 и Feature 2. Их влияние подтверждает сильную зависимость целевой переменной от этих признаков.

2. **Глобальная интерпретация**:  
   Распределение SHAP-значений показало наличие кластеров данных и сложных нелинейных зависимостей. Feature 0 особенно выделяется своей ключевой ролью в определении результата модели.

3. **Локальная интерпретация**:  
   Индивидуальные объяснения для конкретных наблюдений (Force Plot) помогают понять, какие признаки вносят наибольший вклад в предсказания. Dependence Plot для Feature 0 выявил сложные зависимости, которые модель эффективно улавливает.

4. **Заключение**:  
   Модель демонстрирует стабильное поведение с интерпретируемыми результатами. Однако важность Feature 0 требует дополнительного анализа, чтобы понять его влияние в контексте задачи.